In [1]:
from fastapi import FastAPI
from fastapi.params import Body
from fastapi import Response
from pydantic import BaseModel
from typing import Optional
from fastapi import status
from fastapi import HTTPException
import psycopg2.extras

In [2]:
# macros, initials
cuisines = ["Turkish", "Italian", "French", "Lebanese", "Japanese"]

In [3]:
# database connection and initialization
try:
    connection = psycopg2.connect(host='0.0.0.0', database='postgres', 
                user='postgres', password='fastapi24', cursor_factory=psycopg2.extras.RealDictCursor)
    cursor = connection.cursor()
    print("[INFO] Database connected successfully!")
except Exception as databaseException:
    print("[INFO] Connecting to database failed! ", databaseException)

[INFO] Database connected successfully!


In [ ]:
def create_cuisines_table():
    try:
        sql_query = (""" CREATE TABLE IF NOT EXISTS cuisines (
                            cuisine_id SERIAL PRIMARY KEY,
                            cuisine_name VARCHAR(255) NOT NULL
                        )""")
        cursor.execute(sql_query)
    except Exception as e:
        print("[INFO] ", e)

In [ ]:
def insert_cuisines_table():
    sql_query = '''INSERT INTO cuisines (cuisine_name) VALUES (%s)'''
    for cuisine in cuisines:
        cursor.execute(sql_query, (cuisine,))

In [ ]:
create_cuisines_table()
connection.commit()

In [ ]:
insert_cuisines_table()
connection.commit()

In [20]:
def create_foods_table():
    try:
        sql_query = (""" CREATE TABLE IF NOT EXISTS foods (
                            food_id SERIAL PRIMARY KEY,
                            food_name VARCHAR(255) NOT NULL,
                            food_image BYTEA,
                            food_price INTEGER NOT NULL,
                            cuisine_id INTEGER REFERENCES cuisines (cuisine_id)
                        )""")
        cursor.execute(sql_query)
    except Exception as e:
        print("[INFO] ", e)


In [21]:
create_foods_table()
connection.commit()

In [40]:
# spoonacular api setup
# pip3 install spoonacular
import spoonacular as sp

api_key = "971a5d25ea22450db5ba241ebd9881b8"
api = sp.API(api_key)

In [43]:
from PIL import Image
import requests
from io import BytesIO
import openpyxl

In [48]:
connection.commit()
for cuisine in cuisines:
    response = api.search_recipes_complex(cuisine)
    foods = response.json()['results']

    for food in foods:
        # get food price with another call, this call doesnt include price
        food_id = food['id']
        food_info = api.get_recipe_information(food_id, False)
        price = food_info.json()['pricePerServing']

        # title
        name = food['title']

        # image
        image_url = food['image']
        response = requests.get(image_url)
        image = BytesIO(response.content).getbuffer().tobytes()

        query = """INSERT INTO foods (food_name, food_image, food_price, cuisine_id) VALUES (%s, %s, %s, (SELECT cuisine_id FROM cuisines WHERE cuisine_name=%s))"""
        val = (name, image, price, cuisine,)
        cursor.execute(query, val,)
        connection.commit()